<a href="https://colab.research.google.com/github/DAUZEX/Otimiza-o-de-Estoque/blob/main/OTIMIZA%C3%87%C3%83O_ESTOQUE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from datetime import timedelta
import warnings
import os
warnings.filterwarnings('ignore')

# Dados da planilha
data = {
    'EMPRESA': ['FACENS']*27,
    'PRODUTO': ['E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61', 'E3Z-R61',
                'Z-15GW4-B', 'Z-15GW4-B', 'Z-15GW4-B', 'Z-15GW4-B', 'Z-15GW4-B', 'Z-15GW4-B', 'Z-15GW4-B',
                'D4DS-K3', 'D4DS-K3', 'D4DS-K3', 'D4DS-K3', 'D4DS-K3',
                'S8VK-G03024', 'S8VK-G03024',
                'CP1E-N40DT1-D', 'CP1E-N40DT1-D',
                'NB3Q-TW01B', 'NB3Q-TW01B'],
    'MARCA': ['OMRON']*27,
    'QTD': [3, 2, 8, 1, 1, 1, 5, 1, 8, 2, 3, 5, 4, 2, 1, 6, 4, 2, 4, 5, 7, 5, 5, 3, 3, 2, 2],
    'DATA CONSUMO 2023': ['9/21/23', '7/6/23', '12/2/23', '8/2/23', '3/28/23', '1/4/23', '1/11/23', '8/26/23', '1/3/23',
                         '1/1/23', '12/21/23', '3/17/23', '10/29/23', '4/9/23', '8/22/23', '7/16/23',
                         '10/30/23', '3/20/23', '10/25/23', '3/15/23', '4/20/23',
                         '12/7/23', '6/18/23',
                         '12/11/23', '6/1/23',
                         '12/4/23', '6/15/23'],
    'DATA CONSUMO 2024': ['11/14/24', '9/29/24', '8/2/24', '2/15/24', '1/2/24', '1/18/24', '3/7/24', '7/25/24', '7/2/24',
                          '6/20/24', '8/21/24', '9/20/24', '2/1/24', '2/2/24', '2/2/24', '12/7/24',
                          '8/27/24', '1/5/24', '8/19/24', '7/26/24', '3/21/24',
                          '12/14/24', '6/22/24',
                          '12/9/24', '6/30/24',
                          '12/27/24', '6/18/24'],
    'LEAD TIME DO PRODUTO': ['90 DIAS']*9 + ['75 DIAS']*7 + ['30 DIAS']*5 + ['60 DIAS']*2 + ['120 DIAS']*2 + ['30 DIAS']*2
}

# Verificar se os dados foram carregados corretamente
if not data:
    raise ValueError("Erro: Dados de entrada não foram fornecidos.")

# Criar DataFrame
try:
    df = pd.DataFrame(data)
except Exception as e:
    raise ValueError(f"Erro ao criar DataFrame: {e}")

# Função para parsear datas com robustez
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%y', errors='coerce')
    except:
        return pd.to_datetime(date_str, errors='coerce')

# Converter colunas de datas
df['DATA CONSUMO 2023'] = df['DATA CONSUMO 2023'].apply(parse_date)
df['DATA CONSUMO 2024'] = df['DATA CONSUMO 2024'].apply(parse_date)

# Verificar se há datas inválidas
if df[['DATA CONSUMO 2023', 'DATA CONSUMO 2024']].isnull().any().any():
    print("Aviso: Algumas datas não puderam ser convertidas e foram marcadas como NaN.")

# Combinar dados de 2023 e 2024
df_2023 = df[['PRODUTO', 'QTD', 'DATA CONSUMO 2023']].rename(columns={'DATA CONSUMO 2023': 'DATA', 'QTD': 'QUANTIDADE'})
df_2024 = df[['PRODUTO', 'QTD', 'DATA CONSUMO 2024']].rename(columns={'DATA CONSUMO 2024': 'DATA', 'QTD': 'QUANTIDADE'})
df_2023 = df_2023.dropna(subset=['DATA'])
df_2024 = df_2024.dropna(subset=['DATA'])
df_combined = pd.concat([df_2023, df_2024])

# Extrair lead times
lead_times = df[['PRODUTO', 'LEAD TIME DO PRODUTO']].drop_duplicates().set_index('PRODUTO')
lead_times['LEAD TIME DO PRODUTO'] = lead_times['LEAD TIME DO PRODUTO'].str.extract('(\d+)').astype(int)

# Função para prever demanda
def prever_demanda_hibrida(produto, dados):
    df_produto = dados[dados['PRODUTO'] == produto][['DATA', 'QUANTIDADE']].rename(columns={'DATA': 'ds', 'QUANTIDADE': 'y'})
    df_produto = df_produto.groupby('ds').sum().reset_index()
    if len(df_produto) < 2:
        print(f"Aviso: Produto {produto} tem poucos dados para previsão.")
        return pd.DataFrame(columns=['ds', 'yhat'])

    # Prophet
    prophet_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
    prophet_model.fit(df_produto)
    future = prophet_model.make_future_dataframe(periods=365, freq='D')
    prophet_forecast = prophet_model.predict(future)[['ds', 'yhat']]

    # XGBoost
    df_produto['month'] = df_produto['ds'].dt.month
    df_produto['quarter'] = df_produto['ds'].dt.quarter
    X = df_produto[['month', 'quarter']]
    y = df_produto['y']
    xgb_model = XGBRegressor()
    xgb_model.fit(X, y)
    future_features = pd.DataFrame({
        'month': future['ds'].dt.month,
        'quarter': future['ds'].dt.quarter
    })
    xgb_forecast = xgb_model.predict(future_features)

    # Ensemble
    ensemble_forecast = 0.7 * prophet_forecast['yhat'] + 0.3 * xgb_forecast
    forecast_2025 = pd.DataFrame({
        'ds': future['ds'],
        'yhat': ensemble_forecast.clip(0).round()
    })
    return forecast_2025[forecast_2025['ds'].dt.year == 2025][forecast_2025['yhat'] > 0]

# Função para otimizar pedidos
def otimizar_pedidos(produto, forecast, lead_time, custo_pedido=100, custo_armazenamento=5, penalidade_ruptura=50):
    if forecast.empty:
        return pd.DataFrame(columns=['Produto', 'Data_Pedido', 'Data_Demanda', 'Quantidade_Pedido'])

    demandas = forecast['yhat'].values
    horizonte = len(demandas)
    dp = np.zeros(horizonte + 1)
    pedidos = [0] * horizonte

    for t in range(horizonte):
        min_custo = float('inf')
        for q in range(int(max(demandas[t:])) + 1):
            estoque = q - demandas[t]
            custo = custo_pedido * (q > 0) + custo_armazenamento * max(0, estoque)
            if estoque < 0:
                custo += penalidade_ruptura * abs(estoque)
            if t + int(lead_time) < horizonte:
                custo += dp[t + int(lead_time)]
            if custo < min_custo:
                min_custo = custo
                pedidos[t] = q
        dp[t + 1] = min_custo

    return pd.DataFrame({
        'Produto': [produto] * len(forecast),
        'Data_Pedido': forecast['ds'] - timedelta(days=int(lead_time)),
        'Data_Demanda': forecast['ds'],
        'Quantidade_Pedido': pedidos[:len(forecast)]
    })

# Função para agrupar pedidos
def agrupar_pedidos(produtos, lead_times, forecast_dict):
    features = pd.DataFrame({
        'lead_time': [lead_times.loc[p, 'LEAD TIME DO PRODUTO'] for p in produtos],
        'demanda_media': [forecast_dict[p]['yhat'].mean() if not forecast_dict[p].empty else 0 for p in produtos]
    })
    kmeans = KMeans(n_clusters=3, random_state=42)
    clusters = kmeans.fit_predict(features)

    pedidos_agrupados = []
    for cluster in range(3):
        cluster_produtos = [produtos[i] for i in range(len(produtos)) if clusters[i] == cluster]
        if cluster_produtos:
            lead_time_max = max(lead_times.loc[cluster_produtos, 'LEAD TIME DO PRODUTO'])
            for p in cluster_produtos:
                forecast = forecast_dict[p]
                if not forecast.empty:
                    pedidos = otimizar_pedidos(p, forecast, lead_time_max)
                    pedidos_agrupados.append(pedidos)
    return pd.concat(pedidos_agrupados) if pedidos_agrupados else pd.DataFrame()

# Processar produtos
forecast_dict = {}
for produto in df['PRODUTO'].unique():
    forecast = prever_demanda_hibrida(produto, df_combined)
    forecast_dict[produto] = forecast

# Gerar pedidos agrupados
df_pedidos = agrupar_pedidos(df['PRODUTO'].unique(), lead_times, forecast_dict)

# Exibir e salvar
if not df_pedidos.empty:
    print("Resumo das Previsões e Pedidos para 2025:")
    for produto in df_pedidos['Produto'].unique():
        pedidos_produto = df_pedidos[df_pedidos['Produto'] == produto]
        print(f"\nProduto: {produto}")
        print(pedidos_produto[['Data_Pedido', 'Data_Demanda', 'Quantidade_Pedido']].head())

    output_path = os.path.join(os.getcwd(), 'pedidos_2025_avancado.csv')
    try:
        df_pedidos.to_csv(output_path, index=False)
        print(f"\nArquivo '{output_path}' gerado com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")
else:
    print("Nenhum pedido gerado devido a dados insuficientes.")

INFO:prophet:n_changepoints greater than number of observations. Using 13.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu5ah332x/tqhghodj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu5ah332x/b2ab94y3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=61330', 'data', 'file=/tmp/tmpu5ah332x/tqhghodj.json', 'init=/tmp/tmpu5ah332x/b2ab94y3.json', 'output', 'file=/tmp/tmpu5ah332x/prophet_modelchz8ay7b/prophet_model-20250610224214.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
22:42:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:42:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:n_changepoints greater than number of observations. Using 9.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpu5ah332x/i6jovfqq.json
DEBUG:cmdstanpy:input tempfil

Resumo das Previsões e Pedidos para 2025:

Produto: D4DS-K3
    Data_Pedido Data_Demanda  Quantidade_Pedido
139  2024-12-05   2025-01-04                  3
140  2024-12-06   2025-01-05                 12
141  2024-12-07   2025-01-06                 19
142  2024-12-08   2025-01-07                 25
143  2024-12-09   2025-01-08                 29

Produto: CP1E-N40DT1-D
   Data_Pedido Data_Demanda  Quantidade_Pedido
26  2024-09-03   2025-01-01                  3
27  2024-09-04   2025-01-02                  3
28  2024-09-05   2025-01-03                  3
29  2024-09-06   2025-01-04                  3
30  2024-09-07   2025-01-05                  3

Produto: E3Z-R61
   Data_Pedido Data_Demanda  Quantidade_Pedido
65  2024-10-03   2025-01-01                  4
66  2024-10-04   2025-01-02                  4
67  2024-10-05   2025-01-03                  5
68  2024-10-06   2025-01-04                  5
69  2024-10-07   2025-01-05                  6

Produto: Z-15GW4-B
   Data_Pedido Data_Demand